In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pymc3 as pm
import theano.tensor as tt
import weibull
from scipy.stats import exponweib

import os
import shutil
import ctypes
import time


WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
C:\Users\xjl19\Anaconda3\lib\site-packages\theano\configdefaults.py:560: UserWarning: DeprecationWarning: there is no c++ compiler.This is deprecated and with Theano 0.11 a c++ compiler will be mandatory
  warnings.warn("DeprecationWarning: there is no c++ compiler."
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.
C:\Users\xjl19\Anaconda3\lib\site-packages\weibull\__init__.py:9: UserWarning: The "weibull" module is being superceded by the "reliability" module and will no longer be maintained. Please refactor your code for this alternative model and thank you for using 

In [39]:
data = exponweib.rvs(a=1, c=0.7, loc=0, scale=30, size=20)
#  型号2——1
# data = [776.09, 114.93, 145.95, 1058.38, 513.90, 482.00,
#         363.90, 336.04, 337.64, 251.38, 457.58, 596.92,
#          1.75, 104.37, 143.31, 554.34, 639.31, 346.31,
#          219.80, 448.78, 793.57, 332.55, 1007.25, 134.15,
#          85.90, 389.28, 65.43, 249.78, 59.39, 592.12,
#          964.43, 752.49, 475.92, 212.65, 759.03, 289.36,
#          127.02, 41.73, 52.07, 94.24, 272.90, 741.86,
#          657.01, 18.64, 113.62, 105.28, 359.97, 218.97,
#          335.24, 250.72, 45.16, 314.41, 196.28, 16.10,
#          13.99, 242.81, 102.14, 99.69, 60.81, 870.35,
#          524.40, 157.39, 20.90]

# 型号1-1
# data = [776.09, 143.31, 513.90, 157.39, 1007.25, 346.31, 272.90, 475.92, 9.36]

# # 型号2-2
# data = [176.0, 248.00, 10.50, 472.00, 45.00, 39.00, 209.33, 261.25, 510.00, 120.00, 224.00, 348.00, 267.50, 32.00, 50.00, 138.50, 398.00, 353.00, 478.00, 137.06,]

# data = np.array(data)
# data = np.random.exponential(357,10)
data[:10]

array([ 62.68556388,  29.08121036,   1.14988057,  39.77753016,
         6.22721891,  56.43663944,  14.34319967,   5.89392677,
       111.15861708,  38.79326972])

In [17]:
class Translate(pm.distributions.transforms.Transform):
  def __init__(self, v):
    self.v_ = v

  def forward(self, x):
    return x + self.v_

  def forward_val(self, x):
    return x + self.v_

  def backward(self, z):
    return x - self.v_

  def jacobian_det(self, x):
    return 0. * x  # log(1) = 0

  def apply(self, rv):
    return pm.distributions.transforms.TransformedDistribution.dist(rv.distribution, self)

In [2]:
class Nuts_weib():
    def __init__(self,data):
        self.data = data
        
    def classic_estimate(self):
        self.analysis = weibull.Analysis(self.data)
        self.analysis.fit('mle')
#         self.analysis.fit()
 
    def model_flat(self):
        with pm.Model() as model_gibbs:
            a = pm.HalfFlat('a')
            b = pm.HalfFlat('b')
            like = pm.Weibull('observation', a, b, observed=data)
            prior = pm.Potential('prior', -tt.log(a*b))
            
            # inference
            start = pm.find_MAP()
            self.trace_flat = pm.sample(1250,tune=1000, start=start)
    
    def model_gamma(self):            
        with pm.Model() as model_1:
#             l = pm.Gamma('l_pr', alpha=2,beta=1)
            a = pm.Gamma('a',alpha=3 , beta=3)            
            b = pm.Gamma('b', alpha=90 , beta=3)   
#             shift = Translate(l)
#             weib = shift.apply(pm.Weibull('base', alpha=a, beta=b,observed=self.data))
            weib = pm.Weibull('base', alpha=a, beta=b,observed=self.data)   
            start = pm.find_MAP()
            self.trace_gamma = pm.sample(1250,tune=1000,start=start)
        
            

# nuts_weib = Nuts_weib(data)

# # # # nuts_weib.classic_estimate()
# # # # nuts_weib.analysis.fit('mle')
# # # # nuts_weib.analysis.stats


# nuts_weib.model_gamma()
# # a = nuts_weib.trace_gamma

In [ ]:
from time import time

alpha = 0.7
beta = 30
size = 200

start_time = time()

enum_gamma=0
enum_flat=0

li = np.load('s200/gamma_200.npy').tolist()

j_li = []
means_li = []
i = 0
j = 1030
while True:
    end_time = time()
    print('总时长%ds'%(end_time-start_time))
    print('flat错误，%d次'%enum_flat)

    print('第' + str(i+1)+'次')
    try:
        np.random.seed(seed=2500 + li[i])
    except:
        print('使用备用数据%d'%j)
        np.random.seed(seed=2500 + j)
        j += 1
    data = exponweib.rvs(a=1, c=alpha, loc=0, scale=beta, size=size)
    nuts_weib = Nuts_weib(data)
    try:
        nuts_weib.model_flat()
    except:
        i += 1
        enum_gamma += 1
        continue
    print('saving...')
    li.append(i)
    np.save('s_flat_' + str(size) + '/flat_' +str(size)  + '.npy',li)
    pm.trace_to_dataframe(nuts_weib.trace_flat).to_csv('s_flat_' + str(size) + '/flat_' +str(size) + '_' + str(i) + '.csv')
    
    i+= 1
    if len(li) == 1000:
        break

总时长0s
flat错误，0次
第1次


C:\Users\xjl19\Anaconda3\lib\site-packages\pymc3\tuning\starting.py:61: UserWarning: find_MAP should not be used to initialize the NUTS sampler, simply call pymc3.sample() and it will automatically initialize NUTS in a better way.
  warnings.warn('find_MAP should not be used to initialize the NUTS sampler, simply call pymc3.sample() and it will automatically initialize NUTS in a better way.')
logp = -925.74, ||grad|| = 106.17: 100%|███████████████████████████████████████████████| 21/21 [00:00<00:00, 11.32it/s]
Auto-assigning NUTS sampler...
INFO:pymc3:Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
INFO:pymc3:Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
INFO:pymc3:Multiprocess sampling (4 chains in 4 jobs)
NUTS: [b, a]
INFO:pymc3:NUTS: [b, a]
Sampling 4 chains: 100%|████████████████████████████████████████████████████████| 9000/9000 [02:14<00:00, 67.13draws/s]
C:\Users\xjl19\Anaconda3\lib\site-packages\mkl_fft\_nump

saving...
总时长151s
flat错误，0次
第2次


logp = -961.06, ||grad|| = 117.38: 100%|███████████████████████████████████████████████| 21/21 [00:00<00:00, 52.59it/s]
Auto-assigning NUTS sampler...
INFO:pymc3:Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
INFO:pymc3:Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
INFO:pymc3:Multiprocess sampling (4 chains in 4 jobs)
NUTS: [b, a]
INFO:pymc3:NUTS: [b, a]
Sampling 4 chains: 100%|████████████████████████████████████████████████████████| 9000/9000 [02:06<00:00, 70.92draws/s]


saving...
总时长294s
flat错误，0次
第3次


...

logp = -905.66, ||grad|| = 0.0051649: 100%|████████████████████████████████████████████| 23/23 [00:00<00:00, 53.06it/s]
Auto-assigning NUTS sampler...
INFO:pymc3:Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
INFO:pymc3:Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
INFO:pymc3:Multiprocess sampling (4 chains in 4 jobs)
NUTS: [b, a]
INFO:pymc3:NUTS: [b, a]
Sampling 4 chains: 100%|████████████████████████████████████████████████████████| 9000/9000 [02:15<00:00, 66.52draws/s]


saving...
总时长151833s
flat错误，0次
第1051次


logp = -921.39, ||grad|| = 103.84: 100%|███████████████████████████████████████████████| 21/21 [00:00<00:00, 55.15it/s]
Auto-assigning NUTS sampler...
INFO:pymc3:Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
INFO:pymc3:Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
INFO:pymc3:Multiprocess sampling (4 chains in 4 jobs)
NUTS: [b, a]
INFO:pymc3:NUTS: [b, a]
Sampling 4 chains: 100%|████████████████████████████████████████████████████████| 9000/9000 [02:14<00:00, 67.11draws/s]


saving...
总时长151982s
flat错误，0次
第1052次


logp = -943.67, ||grad|| = 103.22: 100%|███████████████████████████████████████████████| 20/20 [00:00<00:00, 53.54it/s]
Auto-assigning NUTS sampler...
INFO:pymc3:Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
INFO:pymc3:Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
INFO:pymc3:Multiprocess sampling (4 chains in 4 jobs)
NUTS: [b, a]
INFO:pymc3:NUTS: [b, a]
Sampling 4 chains:  24%|█████████████▍                                          | 2156/9000 [00:38<01:42, 66.60draws/s]

In [7]:
from time import time
from tqdm import tqdm

alpha = 0.7
beta = 30
size = 200

start_time = time()

enum_gamma=0
enum_flat=0

li = np.load('s200/gamma_200.npy').tolist()
estimates=[]
j_li = []
means_li = []
i = 0
j = 1030
for i in tqdm(li):
    end_time = time()
    np.random.seed(seed=2500 + i)

    data = exponweib.rvs(a=1, c=alpha, loc=0, scale=beta, size=size)
    analysis = weibull.Analysis(data)
    analysis.fit('mle')
    estimates.append([analysis.stats[3],analysis.stats[6]])
    
print('saving...')
df = pd.DataFrame(estimates,columns = ['a','b'])
print(df)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [03:24<00:00,  4.85it/s]


saving...
            a          b
0    0.653745  28.249664
1    0.723958  32.317833
2    0.617179  26.801470
3    0.603130  24.050801
4    0.686223  31.386664
5    0.701773  34.247883
6    0.774921  33.768590
7    0.662945  27.317654
8    0.687348  30.131455
9    0.719143  29.862910
10   0.667411  29.803011
11   0.703860  24.992487
12   0.757847  33.510903
13   0.717535  31.083059
14   0.678413  30.595910
15   0.692305  26.646236
16   0.702394  28.995629
17   0.674509  22.489624
18   0.644077  27.633886
19   0.712654  28.137433
20   0.652330  26.447915
21   0.671994  31.802855
22   0.692537  28.063982
23   0.702212  29.858590
24   0.685744  30.923893
25   0.687260  22.512876
26   0.686364  26.814668
27   0.657058  36.116331
28   0.703980  33.909853
29   0.693376  25.742144
..        ...        ...
970  0.721038  34.449371
971  0.713729  29.321221
972  0.713740  29.156712
973  0.683122  29.131888
974  0.634635  34.299642
975  0.767885  27.810970
976  0.815374  29.729721
977  0.743716  

In [6]:
for i in tqdm(range(1000)):
    pass

TypeError: 'module' object is not callable

In [3]:
li = np.load('s200/gamma_200.npy').tolist()
li

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 169,
 170,
 171,
 173,
 174,
 175,
 176,
 177,
 178,
 180,
 181,
 182,
 183,
 184,
 185,
 186,
 187,
 188,
 189,
 190